# Sentiment Analysis - Twitter, IMDB, YELP

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 104.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 31.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import copy

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/BSAN Projects/NLP_735_Team1/twts_train.csv', encoding='utf-8')
test_df = pd.read_csv('/content/drive/MyDrive/BSAN Projects/NLP_735_Team1/twts_test.csv', encoding='utf-8')

In [ ]:
train_df = train_df.drop(columns=['textID','selected_text'], axis=1)

In [ ]:
test_df = test_df.drop(columns=['textID'], axis=1)

In [ ]:
test = copy.deepcopy(test_df)
train = copy.deepcopy(train_df)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd

# Load the pre-trained roberta model and tokenizer
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Define the function to preprocess and predict sentiment
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    sentiment = torch.argmax(outputs.logits, dim=1).item()
    
    if sentiment == 0:
        return "negative"
    elif sentiment == 1:
        return "neutral"
    else:
        return "positive"

# Predict sentiment for each text in the 'text' dataframe and store it in a new column
test['predicted_sentiment'] = test['text'].apply(predict_sentiment)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
train.dtypes

text         object
sentiment    object
dtype: object

In [ ]:
train['text'] = train['text'].astype(str)

In [ ]:
train['predicted_sentiment'] = train['text'].apply(predict_sentiment)

In [ ]:
# Calculate accuracy by comparing the predicted sentiment values with the true sentiment values in the 'train' dataframe
accuracy = (test['sentiment'] == test['predicted_sentiment']).mean()
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 71.79%


In [ ]:
accuracy = (train['sentiment'] == train['predicted_sentiment']).mean()
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 72.14%


In [ ]:
train.head(15)

,text,sentiment,predicted_sentiment
0,"I`d have responded, if I were going",neutral,neutral
1,Sooo SAD I will miss you here in San Diego!!!,negative,negative
2,my boss is bullying me...,negative,negative
3,what interview! leave me alone,negative,negative
4,"Sons of ****, why couldn`t they put them on t...",negative,negative
5,http://www.dothebouncy.com/smf - some shameles...,neutral,positive
6,2am feedings for the baby are fun when he is a...,positive,positive
7,Soooo high,neutral,neutral
8,Both of you,neutral,neutral
9,Journey!? Wow... u just became cooler. hehe....,positive,positive


# text pre process

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 17.6 MB/s eta 0:00:00


In [ ]:
import contractions
import unicodedata
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

def text_preprocess(text):
    # Convert input to string
    text = str(text)
    
    # Expand contractions
    text = contractions.fix(text)

    # Remove punctuation and stress marks using Unicode
    text = ''.join(c for c in text if not unicodedata.category(c).startswith('P'))
    text = ''.join(c for c in text if not unicodedata.category(c).startswith('Sk'))

    # Tokenize the text
    tokens = word_tokenize(text)

    # Lemmatize words using NLTK
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Remove stopwords, excluding negation words
    negation_words = {'not', 'no', 'n\'t', 'never', 'none', 'nobody', 'nothing', 'neither', 'nowhere', 'cant', 'can\'t', 'dont', 'don\'t'}
    stop_words = set(stopwords.words('english')) - negation_words
    filtered_tokens = [token for token in lemmatized_tokens if isinstance(token, str) and token.lower() not in stop_words]

    # Reconstruct the processed text
    processed_text = ' '.join(filtered_tokens)

    return processed_text

# Example usage
input_text = "This is a sample sentence, which won't be ignored."
output_text = text_preprocess(input_text)
print(output_text)

sample sentence not ignored


In [ ]:
train['preprocessed_text'] = train['text'].apply(text_preprocess) 
test['preprocessed_text'] = test['text'].apply(text_preprocess) 

In [ ]:
train.head()

,text,sentiment,predicted_sentiment,preprocessed_text
0,"I`d have responded, if I were going",neutral,neutral,Id responded going
1,Sooo SAD I will miss you here in San Diego!!!,negative,negative,Sooo SAD miss San Diego
2,my boss is bullying me...,negative,negative,bos bullying
3,what interview! leave me alone,negative,negative,interview leave alone
4,"Sons of ****, why couldn`t they put them on t...",negative,negative,Sons couldnt put release already bought


In [ ]:
tweets = pd.concat([train,test])

# Nested CV

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, f1_score

def train_and_evaluate(X_train_all, y_train_all, X_test_all, y_test_all):
    skf_outer = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
    skf_inner = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

    outer_fold = 5

    for train_index, test_index in skf_outer.split(X_test_all, y_test_all):
        print('Outer fold', outer_fold)

        X_test, y_test = np.array(X_test_all)[test_index], np.array(y_test_all)[test_index]

        inner_fold = 10
        best_inner_score = 0
        best_inner_params = None

        for sub_train_index, dev_index in skf_inner.split(X_train_all, y_train_all):
            print('Inner fold', inner_fold)

            X_train, X_dev = np.array(X_train_all)[sub_train_index], np.array(X_train_all)[dev_index]
            y_train, y_dev = np.array(y_train_all)[sub_train_index], np.array(y_train_all)[dev_index]

            clf = Pipeline([('vect', TfidfVectorizer(smooth_idf=True)),
                            ('clf', LogisticRegression(penalty='l2', solver='liblinear', max_iter=1500, random_state=0))])

            for C in [10]:  # Tune the 'C' hyperparameter
                clf.set_params(clf__C=C)
                clf.fit(X_train, y_train)
                y_dev_pred = clf.predict(X_dev)

                dev_f1_score = f1_score(y_dev, y_dev_pred, average='macro')
                print(f"Inner fold {inner_fold}, C={C}, F1 Score: {dev_f1_score}")

                if dev_f1_score > best_inner_score:
                    best_inner_score = dev_f1_score
                    best_inner_params = {'clf__C': C}

            inner_fold += 1

        print(f"Best inner params for outer fold {outer_fold}: {best_inner_params}")

        # Train the model using the best inner params on the entire train set of the current outer fold
        clf.set_params(**best_inner_params)
        clf.fit(X_train_all, y_train_all)

        y_test_pred = clf.predict(X_test)
        print(classification_report(y_test, y_test_pred))

        outer_fold += 1

# Load your dataset here
# For example, let's assume you have a dataset called `tweets` with preprocessed_text and sentiment columns
X = tweets['text']
y = tweets['predicted_sentiment']

# Split the dataset into train and test sets using train_test_split
X_train_all, X_test_all, y_train_all, y_test_all = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Call the train_and_evaluate function with the input data
train_and_evaluate(X_train_all, y_train_all, X_test_all, y_test_all)


Outer fold 5
Inner fold 10
Inner fold 10, C=10, F1 Score: 0.7096959115453125
Inner fold 11
Inner fold 11, C=10, F1 Score: 0.6902954598085146
Inner fold 12
Inner fold 12, C=10, F1 Score: 0.7055089800524751
Inner fold 13
Inner fold 13, C=10, F1 Score: 0.7011844288394159
Inner fold 14
Inner fold 14, C=10, F1 Score: 0.7070999231747231
Best inner params for outer fold 5: {'clf__C': 10}
              precision    recall  f1-score   support

    negative       0.71      0.70      0.71       570
     neutral       0.61      0.59      0.60       574
    positive       0.78      0.81      0.80       717

    accuracy                           0.71      1861
   macro avg       0.70      0.70      0.70      1861
weighted avg       0.71      0.71      0.71      1861

Outer fold 6
Inner fold 10
Inner fold 10, C=10, F1 Score: 0.7096959115453125
Inner fold 11
Inner fold 11, C=10, F1 Score: 0.6902954598085146
Inner fold 12
Inner fold 12, C=10, F1 Score: 0.7055089800524751
Inner fold 13
Inner fold 13, C

# Trainable embed and Pre-trained GloVe

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

X = tweets['text']
y = tweets['predicted_sentiment']  # predicted sentiment using Bertweet

# Initialize the LabelBinarizer and fit it on the labels
lb = LabelBinarizer()
lb.fit(y)

# Transform the labels into one-hot encoded format
y_one_hot = lb.transform(y)

# Tokenize the text data
max_sequence_length = 100
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_sequences = tokenizer.texts_to_sequences(X)
X_padded = pad_sequences(X_sequences, maxlen=max_sequence_length)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_padded, y_one_hot, test_size=0.2, random_state=42)

# Update vocab_size for the model
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, BatchNormalization, Activation
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers.schedules import ExponentialDecay

# Parameters
embedding_dim = 100
lstm_units = 128
vocab_size = len(tokenizer.word_index) + 1

model_trainable = Sequential()
model_trainable.add(Embedding(vocab_size, embedding_dim, input_length=max_sequence_length))
model_trainable.add(LSTM(lstm_units))
model_trainable.add(Dense(3))  # Replace num_classes with the number of unique labels in your dataset
model_trainable.add(BatchNormalization())
model_trainable.add(Activation('softmax'))

# Compile the model with the specified optimizer settings
initial_learning_rate = 0.001
decay_steps = 1000
decay_rate = 0.9

lr_schedule = ExponentialDecay(initial_learning_rate, decay_steps, decay_rate)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, beta_1=0.9, beta_2=0.999)

model_trainable.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model_trainable.summary()

# Early stopping based on validation accuracy
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3)

# Train the model
history_trainable = model_trainable.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2, callbacks=[early_stopping])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          2861500   
                                                                 
 lstm (LSTM)                 (None, 128)               117248    
                                                                 
 dense (Dense)               (None, 3)                 387       
                                                                 
 batch_normalization (BatchN  (None, 3)                12        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 3)                 0         
                                                                 
Total params: 2,979,147
Trainable params: 2,979,141
Non-trainable params: 6
______________________________________________

In [ ]:
# Convert the predicted probabilities into class labels
y_pred_trainable = np.argmax(model_trainable.predict(X_test), axis=-1)

# Convert the true labels (y_test) back to their original format if needed
y_test_original = np.argmax(y_test, axis=-1)

# Replace the class names in the list with your own class names
class_names = ['negative', 'neutral', 'positive']

# Print the classification report
print("\nClassification report for trainable embeddings model:")
print(classification_report(y_test_original, y_pred_trainable, target_names=class_names))

194/194 [==============================] - 1s 3ms/step

Classification report for trainable embeddings model:
              precision    recall  f1-score   support

    negative       0.68      0.70      0.69      1919
     neutral       0.61      0.45      0.52      1946
    positive       0.69      0.83      0.75      2338

    accuracy                           0.67      6203
   macro avg       0.66      0.66      0.66      6203
weighted avg       0.66      0.67      0.66      6203



# GloVe

# Load GloVe embeddings and create embedding matrix

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

X = tweets['text']
y = tweets['predicted_sentiment']

lb = LabelBinarizer()
lb.fit(y)

y_one_hot = lb.transform(y)

max_sequence_length = 100
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_sequences = tokenizer.texts_to_sequences(X)
X_padded = pad_sequences(X_sequences, maxlen=max_sequence_length)

X_train, X_test, y_train, y_test = train_test_split(X_padded, y_one_hot, test_size=0.2, random_state=42)

vocab_size = len(tokenizer.word_index) + 1

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, BatchNormalization, Activation
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers.schedules import ExponentialDecay

# Load GloVe embeddings
def load_glove_embeddings(file_path, embedding_dim, word_index):
    embeddings_index = {}
    with open('/content/drive/MyDrive/BSAN Projects/glove.6B.100d.txt', 'r', encoding='utf8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs

    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

    return embedding_matrix

embedding_dim = 100
glove_path = '/content/drive/MyDrive/BSAN Projects/glove.6B.100d.txt'  # Replace with the path to your GloVe file
embedding_matrix = load_glove_embeddings(glove_path, embedding_dim, tokenizer.word_index)

lstm_units = 128

model_glove = Sequential()
model_glove.add(Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=max_sequence_length, trainable=False))
model_glove.add(LSTM(lstm_units))
model_glove.add(Dense(3))
model_glove.add(BatchNormalization())
model_glove.add(Activation('softmax'))

In [ ]:
initial_learning_rate = 0.001
decay_steps = 1000
decay_rate = 0.9

lr_schedule = ExponentialDecay(initial_learning_rate, decay_steps, decay_rate)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, beta_1=0.9, beta_2=0.999)

model_glove.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model_glove.summary()

early_stopping = EarlyStopping(monitor='val_accuracy', patience=3)

history_glove = model_glove.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2, callbacks=[early_stopping])

y_pred_glove = np.argmax(model_glove.predict(X_test), axis=-1)
y_test_original = np.argmax(y_test, axis=-1)

class_names = ['negative', 'neutral', 'positive']

print("\nClassification report for GloVe embeddings model:")
print(classification_report(y_test_original, y_pred_glove, target_names=class_names))


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 100)          2861500   
                                                                 
 lstm_1 (LSTM)               (None, 128)               117248    
                                                                 
 dense_1 (Dense)             (None, 3)                 387       
                                                                 
 batch_normalization_1 (Batc  (None, 3)                12        
 hNormalization)                                                 
                                                                 
 activation_1 (Activation)   (None, 3)                 0         
                                                                 
Total params: 2,979,147
Trainable params: 117,641
Non-trainable params: 2,861,506
______________________________________

# Trainable embed and Avg. Pooling

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

X = tweets['text']
y = tweets['predicted_sentiment']  # predicted sentiment using Bertweet

# Initialize the LabelBinarizer and fit it on the labels
lb = LabelBinarizer()
lb.fit(y)

# Transform the labels into one-hot encoded format
y_one_hot = lb.transform(y)

# Tokenize the text data
max_sequence_length = 100
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_sequences = tokenizer.texts_to_sequences(X)
X_padded = pad_sequences(X_sequences, maxlen=max_sequence_length)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_padded, y_one_hot, test_size=0.2, random_state=42)

# Update vocab_size for the model
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, BatchNormalization, Activation, GlobalAveragePooling1D, Bidirectional, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers.schedules import ExponentialDecay

# Parameters
embedding_dim = 100
lstm_units = 128
vocab_size = len(tokenizer.word_index) + 1

model_trainable = Sequential()
model_trainable.add(Embedding(vocab_size, embedding_dim, input_length=max_sequence_length))
model_trainable.add(Bidirectional(LSTM(lstm_units, return_sequences=True)))
model_trainable.add(GlobalAveragePooling1D())
model_trainable.add(Dropout(0.5))
model_trainable.add(Dense(3))  # Replace num_classes with the number of unique labels in your dataset
model_trainable.add(BatchNormalization())
model_trainable.add(Activation('softmax'))


# Compile the model with the specified optimizer settings
initial_learning_rate = 0.001
decay_steps = 1000
decay_rate = 0.9

lr_schedule = ExponentialDecay(initial_learning_rate, decay_steps, decay_rate)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, beta_1=0.9, beta_2=0.999)

model_trainable.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model_trainable.summary()

# Early stopping based on validation accuracy
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3)

# Train the model
history_trainable = model_trainable.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2, callbacks=[early_stopping])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 100, 100)          2861500   
                                                                 
 bidirectional (Bidirectiona  (None, 100, 256)         234496    
 l)                                                              
                                                                 
 global_average_pooling1d (G  (None, 256)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 3)                 771       
                                                                 
 batch_normalization_2 (Batc  (None, 3)               

In [ ]:
# Convert the predicted probabilities into class labels
y_pred_trainable = np.argmax(model_trainable.predict(X_test), axis=-1)

# Convert the true labels (y_test) back to their original format if needed
y_test_original = np.argmax(y_test, axis=-1)

# Replace the class names in the list with your own class names
class_names = ['negative', 'neutral', 'positive']

# Print the classification report
print("\nClassification report for trainable embeddings model:")
print(classification_report(y_test_original, y_pred_trainable, target_names=class_names))

194/194 [==============================] - 1s 4ms/step

Classification report for trainable embeddings model:
              precision    recall  f1-score   support

    negative       0.73      0.67      0.70      1919
     neutral       0.59      0.66      0.62      1946
    positive       0.78      0.76      0.77      2338

    accuracy                           0.70      6203
   macro avg       0.70      0.70      0.70      6203
weighted avg       0.71      0.70      0.70      6203

